In [ ]:
# This script prepares geotiffs of vegetation indices for pixel-wise trend fitting

# import libraries
import ic_wrapper as wp # type: ignore (current workaround for pylance missing imports issue; doesn't affect functionality)
import trend_fitting as tf # type: ignore 
import recovery_metrics as rm # type: ignore 
import preliminary_values as pv # type: ignore
import matplotlib.pyplot as plt
import numpy as np
import os 
from glob import glob

In [ ]:
# ======================
# Paths: change paths accordingly
# ======================
data_dir = "/Volumes/GoogleDrive/My Drive/GEE_EXPORTS/volcanic_veg"
data_dir2 = "/Volumes/GoogleDrive/My Drive/GEE_EXPORTS/volcanic_veg_test"

# folders to get input data  
geotiff_input_path = os.path.join(data_dir, 'geotiff_files')

# veg recovery directory
veg_recovery_dir = os.path.join(data_dir2, 'vegrecovery_outputs_test')

# folder to keep results from trend fitting
trend_recovery_dir = os.path.join(data_dir2, 'trend_results_test')

# make folders if folders currently do not exist
path_list = [data_dir, geotiff_input_path, veg_recovery_dir, trend_recovery_dir]
for path in path_list: 
    exist = os.path.exists(path)
    if not exist: 
        os.makedirs(path)
        print(f"new {path} folder created!")
    else: 
        print(f"{path} already exists! No folder was created")

In [ ]:
# ======================
# Read in geotiffs
# ======================
file_list_NBR = glob(geotiff_input_path + '/*_NBR.tif')
# file_list_NDVI = glob(geotiff_input_path + '/*_NDVI.tif')
# file_list_SAVI = glob(geotiff_input_path + '/*_SAVI.tif')

In [ ]:
# ======================
# ingest and clean data
# ======================
valid_NBR_withyears = wp.wrapper_clean_ingest(file_list_NBR, 'NBR')

In [ ]:
# ======================
# Perform pixel-wise linear-log trend fitting
# ======================
nbr_fit_result = tf.trend_fit(valid_NBR_withyears)
# ndvi_fit_result = tf.trend_fit(valid_NDVI_withyears)
# savi_fit_result = tf.trend_fit(valid_SAVI_withyears)

In [ ]:
# ======================
# Obtain dVIs
# ======================
dNBR_col = pv.get_dVI(valid_NBR_withyears)
# dNDVI_col = pv.get_dVI(valid_NDVI_withyears)

In [ ]:
# ======================
# Obtain recovery metrics from fitted linear-log curves
# ======================
# absolute measure of post-disturbance regrowth (5 years)
nbr_abs_regrowth = rm.abs_regrowth(nbr_fit_result, 5)
# ndvi_abs_regrowth = rm.abs_regrowth(ndvi_fit_result, 5)

# relative measure of post-disturbance regrowth (RI)
nbr_RI = rm.rel_regrowth(nbr_fit_result, nbr_abs_regrowth, dNBR_col)
# ndvi_RI = rm.rel_regrowth(ndvi_fit_result, ndvi_abs_regrowth, dNDVI_col)

# slope
slope_nbr = rm.get_slope(nbr_fit_result)
# slope_ndvi = rm.get_slope(ndvi_fit_result)

# number of years to certain % recovery
# 20% recovery
year20_nbr = rm.numyears_from_trend(valid_NBR_withyears, nbr_fit_result, 0.2)
year20_nbr = np.where(year20_nbr > 0, year20_nbr, np.nan) # filter for very negative values

# year20_ndvi = rm.numyears_from_trend(valid_NDVI_withyears, ndvi_fit_result, 0.2)
# year20_ndvi = np.where(year20_ndvi > 0, year20_ndvi, np.nan)

# 80% recovery
year80_nbr = rm.numyears_from_trend(valid_NBR_withyears, nbr_fit_result, 0.8)
year80_nbr = np.where(year80_nbr > 0, year80_nbr, np.nan) # filter for very negative values

# year80_ndvi = rm.numyears_from_trend(valid_NDVI_withyears, ndvi_fit_result, 0.8)
# year80_ndvi = np.where(year80_ndvi > 0, year80_ndvi, np.nan)